In [1]:

import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [2]:
#ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        validation_split=0.2,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(validation_split=0.2,
                                        rescale=1./255)



In [3]:

# Create generators to read images from dataset directory
# -------------------------------------------------------
cwd=os.getcwd()
dataset_dir='/kaggle/input/ann-and-dl-image-classification/Classification_Dataset'
# img

# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256


class_list=['owl',
            'galaxy',
            'lightning',
            'wine-bottle',
            't-shirt',
            'waterfall',
            'sword',
            'school-bus',
            'calculator',
            'sheet-music',
            'airplanes',
            'lightbulb',
            'skyscraper',
            'mountain-bike',
            'fireworks',
            'computer-monitor',
            'bear',
            'grand-piano',
            'kangaroo',
            'laptop']


training_dir = os.path.join(dataset_dir, 'training')
# Training
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               classes=class_list,
                                               color_mode='rgb',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')  # targets are directly converted into one-hot vectors
#Validation
valid_gen = train_data_gen.flow_from_directory(training_dir,
                                             batch_size=bs,
                                             class_mode='categorical',
                                             classes=class_list,
                                             color_mode='rgb',
                                             shuffle=True,
                                             seed=SEED,
                                             subset='validation')

Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.


In [4]:
#Create dataset object for
num_classes=20

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),                                   
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# repeat:
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()


# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# repeat:
valid_dataset = valid_dataset.repeat()

In [5]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

58892288/58889256 [==============================] - 5s 0us/step


In [6]:
vgg.summary()
vgg.layers

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [7]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               16777728  
_________________________________________________________________
dense_1 (Dense)              (None, 20)                10260     
Total params: 31,502,676
Trainable params: 23,867,412
Non-trainable params: 7,635,264
_________________________________________________________________


In [8]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [9]:
callbacks=[]
# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 39 steps, validate for 10 steps
Epoch 1/100
39/39 [==============================] - 840s 22s/step - loss: 3.5501 - accuracy: 0.0569 - val_loss: 2.9927 - val_accuracy: 0.0651
Epoch 2/100
39/39 [==============================] - 830s 21s/step - loss: 2.9960 - accuracy: 0.0634 - val_loss: 2.9880 - val_accuracy: 0.0651
Epoch 3/100
39/39 [==============================] - 809s 21s/step - loss: 2.9881 - accuracy: 0.0642 - val_loss: 2.9842 - val_accuracy: 0.0651
Epoch 4/100
39/39 [==============================] - 807s 21s/step - loss: 2.9849 - accuracy: 0.0561 - val_loss: 2.9823 - val_accuracy: 0.0651
Epoch 5/100
39/39 [==============================] - 814s 21s/step - loss: 2.9823 - accuracy: 0.0642 - val_loss: 2.9795 - val_accuracy: 0.0651
Epoch 6/100
39/39 [==============================] - 807s 21s/step - loss: 2.9804 - accuracy: 0.0642 - val_loss: 2.9750 - val_accuracy: 0.0651
Epoch 7/100
39/39 [==============================] - 801s 21s/step - loss: 2.9795 - accuracy: 0.0642